In [1]:
import pandas as pd
import numpy as np
import glob
import math
import json
import requests
from geopy.geocoders import Nominatim

the home sales part

In [2]:
# create a list of zip code which would be the name of communities as well
zip_code=['20001','20002','20003','20004','20005','20006','20007','20008','20009','20010','20011','20012',
         '20015','20016','20017','20018','20019','20020','20024','20032','20036','20037']

In [3]:
areas=pd.read_csv("../Area.csv",dtype={"Zip Code":str})

In [4]:
pop=areas["Population"]

In [5]:
pop=pop.map(lambda x: x.replace(",","")).astype(float)

In [6]:
# read all files in one folder
path_Sales =r'../Home Sales/9_1_2018-9_1_2019, Home Sales by Zip' 
allFiles_Sales = glob.glob(path_Sales + "/*.csv")
frame_Sales = pd.DataFrame()
list_Sales = []
for file_Sales in allFiles_Sales:
    df = pd.read_csv(file_Sales,index_col=None, header=0)
    list_Sales.append(df)

for i in range(len(list_Sales)):
    list_Sales[i]["Zip Code"]=zip_code[i]
    
frame_Sales=pd.concat(list_Sales)

In [7]:
frame_Sales.to_csv("sales.csv")

In [8]:
file_names=[]
for i in range (0,22):
    file_names.append('zip_'+zip_code[i])

files={}
for i in range(0,22):
    files[file_names[i]]=list_Sales[i]

In [9]:
# create a list of average rental amount
ave_rent=[]
for i in range(0,22):
    ave_rent.append(list_Sales[i].groupby('Beds', as_index=False)['Current Price'].mean())

In [10]:
for i in range(0,22):
    ave_rent[i]['Zip Code']=zip_code[i]
    
data=pd.DataFrame()
for i in range(0,22):
    data=data.append(ave_rent[i])
    
data.rename(columns={'Current Price':'Ave Rent'},inplace=True)

In [11]:
# create a dataframe to store data 
df=pd.DataFrame(zip_code,columns={"Zip Code"})
df["Population"]=pop


In [12]:
# create a dataframe of home sale prices for different number of beds
bed_no=np.arange(0,8)

df_s=[]
for i in range(len(bed_no)):
    df_s.append(data[data["Beds"]==bed_no[i]])

names=[]
for i in range(len(bed_no)):
    names.append('Price for Sale|'+str(int(bed_no[i]))+" Beds")

for i in range(len(bed_no)):
    df_s[i].rename(columns={'Ave Rent':names[i]}, inplace=True)
    del df_s[i]["Beds"]

C:\Users\19018\Anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [13]:
for i in range(len(bed_no)):
    df=pd.merge(df,df_s[i],how="left",on="Zip Code")

In [14]:
df.rename(columns={"Price for Sale|0 Beds":"Price for Sale|Studio","Price for Sale|1 Beds":"Price for Sale|1 Bed"}, inplace=True)

In [15]:
# class Community:
#     def __init__(self,zipcode):
#         self.zipcode=zipcode
#     def Ave_Rent(self,Beds):
#         ave_rent=df.loc[(df["Zip_Code"]==zipcode) & (df["Beds"]==Beds)]['Ave_Rent']

# zip_20001=Community(20001)

    

In [16]:
# create a function to rank the performance
def ranking(data):
    data2=data.drop_duplicates(keep='first', inplace=False)
    return data.map(lambda x:5 if x>np.nanpercentile(data2,80)
                               else (4 if np.nanpercentile(data2,60)<x<=np.nanpercentile(data2,80)
                               else (3 if np.nanpercentile(data2,40)<x<=np.nanpercentile(data2,60) 
                               else (2 if np.nanpercentile(data2,20)<x<=np.nanpercentile(data2,40)
                               else (1)))))

home appreciation part

In [17]:
path_09 =r'../Home Sales/9_1_2008-9_1_2009 Home Sales by Zip' 
allFiles_09 = glob.glob(path_09 + "/*.csv")
frame_09 = pd.DataFrame()
list_09 = []
for file_Sales in allFiles_09:
    df_09 = pd.read_csv(file_Sales,index_col=None, header=0,dtype={'Zip':str})
    df_09["Current Price"]=df_09["Current Price"].map(lambda x: str.split(x,"$")[-1].replace(",",""))
    df_09["Current Price"]=df_09["Current Price"].astype(float)
    df_09=df_09.rename(columns={"Current Price":"2009"})
    list_09.append(df_09)
frame_09=pd.concat(list_09)

C:\Users\19018\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [18]:
ave_09=pd.DataFrame(frame_09.groupby(by=["Zip","Beds"])["2009"].mean())
ave_09=ave_09.reset_index(["Zip","Beds"])

In [19]:
path_14 =r'../Home Sales/9_1_2013-9_1_2014 Home Sales by Zip' 
allFiles_14 = glob.glob(path_14 + "/*.csv")
frame_14 = pd.DataFrame()
list_14 = []
for file_Sales in allFiles_14:
    df_14 = pd.read_csv(file_Sales,index_col=None, header=0,dtype={'Zip':str})
    df_14["Current Price"]=df_14["Current Price"].astype(str)
    df_14["Current Price"]=df_14["Current Price"].map(lambda x: str.split(x,"$")[-1].replace(",",""))
    df_14["Current Price"]=df_14["Current Price"].astype(float)
    df_14=df_14.rename(columns={"Current Price":"2014"})
    list_14.append(df_14)
frame_14=pd.concat(list_14)

C:\Users\19018\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


In [20]:
ave_14=pd.DataFrame(frame_14.groupby(by=["Zip","Beds"])["2014"].mean())
ave_14=ave_14.reset_index(["Zip","Beds"])

In [21]:
ave_19=pd.DataFrame(frame_Sales.groupby(by=["Zip Code","Beds"])["Current Price"].mean())
ave_19=ave_19.reset_index(["Zip Code","Beds"])
ave_19=ave_19.rename(columns={"Zip Code":"Zip","Current Price":"2019"})

In [22]:
ave=pd.merge(ave_09,ave_14,on=["Zip","Beds"])
ave=pd.merge(ave,ave_19,on=["Zip","Beds"])

In [23]:
ave["10 years appreciation"]=((ave["2019"]-ave["2009"])/ave["2009"]).apply(lambda x: format(x, '.2%'))

In [24]:
ave["5 years appreciation"]=((ave["2019"]-ave["2014"])/ave["2014"]).apply(lambda x: format(x, '.2%'))

In [25]:
ave.to_csv("sales by years.csv")

In [26]:
dfsale=pd.DataFrame(data=zip_code,columns={"Zip"},dtype=str)
dfapp=pd.DataFrame(data=zip_code,columns={"Zip"},dtype=str)
for i in range(8):
    ave2=ave[ave["Beds"]==i].reset_index(drop=True)
    ave2=ave2.drop(columns="Beds")
    ave2=ave2.merge(dfsale,how="right",on="Zip")
    app5=[]
    app10=[]
    name1=str("5 years appreciation|"+'%d beds'%(i))
    name2=str("10 years appreciation|"+'%d beds'%(i))
    for j in range(len(zip_code)):
        if len(ave2[ave2["Zip"]==zip_code[j]]["5 years appreciation"])==0:
            app5.append(" ")
        else:
            app5.append(ave2[ave2["Zip"]==zip_code[j]]["5 years appreciation"].iloc[0])
        if len(ave2[ave2["Zip"]==zip_code[j]]["10 years appreciation"])==0:
            app10.append(" ")
        else:
            app10.append(ave2[ave2["Zip"]==zip_code[j]]["10 years appreciation"].iloc[0])
    ave2[name1]=app5
    ave2[name2]=app10
    ave2=ave2.drop(columns={"2009","2014","2019","5 years appreciation","10 years appreciation"})
    dfapp=dfapp.merge(ave2,on="Zip",how="right")

In [27]:
dfapp=dfapp.rename(columns={"5 years appreciation|0 beds":"5 years appreciation|Studio",
                            "10 years appreciation|0 beds":"10 years appreciation|Studio",
                           "Zip":"Zip Code","5 years appreciation|1 beds":"5 years appreciation|1 bed",
                           "10 years appreciation|1 beds":"10 years appreciation|1 bed"})

In [28]:
dfapp.to_csv("home sales appreciation.csv")

In [29]:
df=df.merge(dfapp,on="Zip Code")

Home Rental part

In [30]:
rent=pd.read_csv("../Rentals/Rentals.csv",dtype={"Zip Code": str})

In [31]:
df=pd.merge(df,rent,on="Zip Code")

the elementary school part

In [32]:
# read the elementary school data
elm_sch=pd.read_csv('../Public School Education Performance/DCPS Elementary.csv')

In [33]:
per_gro=elm_sch["Perf_Growth"]
elm_sch["Performance of Growth"]=ranking(per_gro)

In [34]:
per_safe=elm_sch["Perf_Safe"]
per_safe=per_safe.astype(float)
elm_sch["Performance of Safety"]=ranking(per_safe)

In [35]:
per_teacher=elm_sch["Perf_Teacher"]
elm_sch["Performance of Teachers"]=ranking(per_teacher)

In [36]:
per_enga=elm_sch["Perf_Engage_Current_Year"]
elm_sch["Performance of Engage"]=ranking(per_enga)

In [37]:
elm_sch["Dir_Zip"]=elm_sch["Dir_Zip"].astype(str)

In [38]:
perf_elm=[]
for i in range(len(elm_sch)):
    data=elm_sch.iloc[i]
    su_m=(data["Performance of Teachers"]+data["Performance of Engage"]+
                              data["Performance of Safety"]+data["Performance of Growth"])
    num=((int(np.isnan(data["Performance of Teachers"])==False))+int((np.isnan(data["Performance of Engage"])==False))+
         (int(np.isnan(data["Performance of Safety"])==False))+int((np.isnan(data["Performance of Growth"])==False)))
    score=su_m/num
    perf_elm.append(score)
elm_sch["Performance"]=perf_elm

In [39]:
per_of_elm_sch=[]
for i in range(len(zip_code)):
    per_of_elm_sch.append(round(np.mean(elm_sch[elm_sch["Dir_Zip"].astype(str)==zip_code[i]]["Performance"]),3))

In [40]:
# add the performance of elementary school to dataframe
df["Performance of Elementary School"]=per_of_elm_sch
df["Performance of Elementary School"]=ranking(df["Performance of Elementary School"])

In [41]:
elm_sch["Ranking"]=ranking(elm_sch['Performance'])

In [42]:
good_elm=elm_sch.sort_values(by="Performance",ascending=False)[:5]

In [43]:
good_elm.to_csv("good elm sch.csv")

the middle school part

In [44]:
mid_sch=pd.read_csv('../Public School Education Performance/DCPS Middle School.csv')

In [45]:
per_gro=mid_sch["Perf_Growth"]
mid_sch["Performance of Growth"]=ranking(per_gro)

In [46]:
per_safe=mid_sch["Perf_Safety"]
mid_sch["Performance of Safety"]=ranking(per_safe)


In [47]:
per_enga=mid_sch["Perf_Engage_Current_Year"]
mid_sch["Performance of Engage"]=ranking(per_enga)

In [48]:
per_teacher=mid_sch["Perf_Teachers_Current_Year"]
mid_sch["Performance of Teachers"]=ranking(per_teacher)

In [49]:
perf_mid=[]
for i in range(len(mid_sch)):
    data=mid_sch.iloc[i]
    su_m=(data["Performance of Teachers"]+data["Performance of Engage"]+
                              data["Performance of Safety"]+data["Performance of Growth"])
    num=((int(np.isnan(data["Performance of Teachers"])==False))+int((np.isnan(data["Performance of Engage"])==False))+
         (int(np.isnan(data["Performance of Safety"])==False))+int((np.isnan(data["Performance of Growth"])==False)))
    score=su_m/num
    perf_mid.append(score)
mid_sch["Performance"]=perf_mid


In [50]:
per_of_mid_sch=[]
for i in range(len(zip_code)):
    per_of_mid_sch.append(round(np.mean(mid_sch[mid_sch["Dir_Zip"].astype(str)==zip_code[i]]["Performance"]),3))

In [51]:
# add the performance of middle school to dataframe
df["Performance of Middle School"]=per_of_mid_sch
df["Performance of Middle School"]=ranking(df["Performance of Middle School"])

In [52]:
mid_sch["Ranking"]=ranking(mid_sch["Performance"])

In [53]:
good_mid=mid_sch.sort_values(by="Performance",ascending=False)[:5]

In [54]:
good_mid.to_csv("good middle school.csv")

the high school part

In [55]:
# read the high school data
hig_sch=pd.read_csv('../Public School Education Performance/High School SAT Scores Washington, DC.csv',header=5)
hig_sch=hig_sch.dropna()

for i in range(len(hig_sch["Zip Code"])):
    hig_sch=hig_sch.replace(hig_sch["Zip Code"][i],str(int(hig_sch["Zip Code"][i])))

In [56]:
hig_sch_sub=pd.read_csv('../Public School Education Performance/DCPS 2018 Graduation Data Overall Subgroup.csv',header=5)

In [57]:
hig_grad=pd.DataFrame(hig_sch_sub[hig_sch_sub['Subgroup']=='All'])
hig_grad.reset_index(drop=True,inplace=True)

In [58]:
highsch=pd.merge(hig_sch,hig_grad,how='right',on=["School Code","School Name"])
highsch.drop(19,inplace=True)

In [59]:
per_grad=highsch["% Grad"]

highsch["Performance of Graduation"]=ranking(per_grad)
highsch['Total Average'] = highsch['Total Average'].astype(float)

per_sat=highsch["Total Average"]

highsch["Performance of SAT"]=ranking(per_sat)

perf_high=[]
for i in range(len(highsch)):
    data=highsch.iloc[i]
    su_m=(data["Performance of SAT"]+data["Performance of Graduation"])
    num=((int(np.isnan(data["Performance of SAT"])==False))+int((np.isnan(data["Performance of Graduation"])==False)))
    score=su_m/num
    perf_high.append(score)
highsch["Performance"]=perf_high


per_of_high_sch=[]
for i in range(len(zip_code)):
    per_of_high_sch.append(round(np.mean(highsch[highsch["Zip Code"]==zip_code[i]]["Performance"]),3))

In [60]:
# add the performance of high school to dataframe
df["Performance of High School"]=per_of_high_sch
df["Performance of High School"]=ranking(df["Performance of High School"])

In [61]:
highsch["Ranking"]=ranking(highsch["Performance"])

In [62]:
good_hig=highsch.sort_values(by="Performance",ascending=False)[:5]

In [63]:
good_hig.to_csv("good high school.csv")

university part

In [64]:
uni=pd.read_csv("../Public School Education Performance/DC University.csv",dtype={"Zip":str})

In [65]:
unis=pd.DataFrame(zip_code,columns={"Zip Code"})

In [66]:
unis["Schools"]=None

In [67]:
for i in range(len(zip_code)):
    un=uni[uni["Zip"]==zip_code[i]]["School"]
    unis["Schools"][i]=un

In [68]:
df=df.merge(unis,on="Zip Code")

the train station part

In [69]:
train=pd.read_csv("../Transit Stations/metro_stations.csv")
train=train[train["State"]=="DC"]
train["Zip"]=train["Zip"].astype(str)

In [70]:
dens=[]
for i in range(len(zip_code)):
    dens.append((train[train["Zip"]==zip_code[i]]["Station"].count()))

In [71]:
df["Availability of Trains"]=dens/df["Population"]
df["Availability of Trains"]=ranking(df["Availability of Trains"])

hospital part

In [72]:
hos=pd.read_csv("../Hospitals/hista_dc_hospitals_tbl.csv")
hos["Zip Code"]=hos["Zip Code"].astype(str)

In [73]:
dens=[]
for i in range(len(zip_code)):
    dens.append((hos[hos["Zip Code"]==zip_code[i]]["Code"].count()))

In [74]:
df["Availability of Hospitals"]=dens/df["Population"]
df["Availability of Hospitals"]=ranking(df["Availability of Hospitals"])

In [75]:
beds=[]
for i in range(len(zip_code)):
    beds.append(hos[hos["Zip Code"]==zip_code[i]]["Beds"].sum())

In [76]:
df["Availability of Hospital Beds"]=beds/df["Population"]
df["Availability of Hospital Beds"]=ranking(df["Availability of Hospital Beds"])

child care part

In [77]:
cc=pd.read_csv("../Daily Needs/September 2019 Child Development Facility Listing.csv")

In [78]:
cc["License Expiration"]=pd.to_datetime(cc["License Expiration"],format="%m/%d/%Y")

In [79]:
cc=cc[cc["License Expiration"]>pd.datetime.now()]

In [80]:
cc["Zip Code"]=cc["Physical Address"].map(lambda x: str.split(x," ")[-1])

In [81]:
dens=[]
for i in range(len(zip_code)):
    dens.append(cc[cc["Zip Code"]==zip_code[i]]["License Number"].count())

In [82]:
df["Availability of Child Care"]=dens/df["Population"]
df["Availability of Child Care"]=ranking(df["Availability of Child Care"])

In [83]:
cap=cc["Authorized Capacity"]

In [84]:
caps=[]
for i in range(len(zip_code)):
    caps.append(cc[cc["Zip Code"]==zip_code[i]]["Authorized Capacity"].sum())

In [85]:
df["Capacity of Child Care"]=cap/df["Population"]
df["Capacity of Child Care"]=ranking(df["Capacity of Child Care"])

In [86]:
df["Performance of Child Care"]=ranking(df["Capacity of Child Care"]+df["Availability of Child Care"])

library part

In [87]:
lib=pd.read_csv("../Library/hista_dc_libraries_tbl.csv")
lib["Zip"]=lib["Zip"].astype(str)

In [88]:
dens=[]
for i in range(len(zip_code)):
    dens.append((lib[lib["Zip"]==zip_code[i]]["Library Name"].count()))

In [89]:
df["Availability of Libraries"]=dens/df["Population"]
df["Availability of Libraries"]=ranking(df["Availability of Libraries"])

grocery part

In [90]:
url="https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Business_and_Economic_Development_WebMercator/MapServer/4/query?where=1%3D1&outFields=*&outSR=4326&f=json"

In [91]:
r=requests.post(url)

In [92]:
a=json.loads(r.text)

In [93]:
g=[]
for i in range(len(a["features"])):
    g.append(a["features"][i]["attributes"])

In [94]:
groc=pd.DataFrame(g)
groc["ZIPCODE"]=groc["ZIPCODE"].astype(str)

In [95]:
#groc=pd.read_csv("../Daily Needs/Grocery_Store_Locations.csv",dtype={'ZIPCODE':str})
names=pd.unique(groc["STORENAME"])
ranks=pd.read_csv("../Daily Needs/Grocery.csv")

groc=pd.merge(groc,ranks,how="left",on="STORENAME")

where_are_nan = np.isnan(groc["RANKING"])
groc["RANKING"][where_are_nan]=1


C:\Users\19018\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [96]:
barg=pd.read_csv("../Daily Needs/bargain.csv")
barg["Rank"]=ranking(barg["Score"])

groc=pd.merge(groc,barg,how="left",on="STORENAME")

In [97]:
ave_rank=[]
num_s=[]
for i in range(len(zip_code)):
    num_s.append(groc[groc["ZIPCODE"]==zip_code[i]]["STORENAME"].count())
    ave_rank.append(groc[groc["ZIPCODE"]==zip_code[i]]["RANKING"].mean())

In [98]:
barank=[]
for i in range(len(zip_code)):
    barank.append(groc[groc["ZIPCODE"]==zip_code[i]]["Rank"].mean())

In [99]:
grocery=pd.DataFrame(zip_code)

grocery["Ranking of Grocery"]=ave_rank
grocery["Ranking of Grocery"]=ranking(grocery["Ranking of Grocery"])

grocery["Density of Grocery"]=num_s/df["Population"]
grocery["Density of Grocery"]=ranking(grocery["Density of Grocery"])

grocery["Ranking of Bargain"]=barank
grocery["Ranking of Bargain"]=ranking(grocery["Ranking of Bargain"])

In [100]:
df["Availability of Grocery|Freshness"]=grocery["Ranking of Grocery"]+grocery["Density of Grocery"]
df["Availability of Grocery|Freshness"]=ranking(df["Availability of Grocery|Freshness"])

df["Availability of Grocery|Bargain"]=grocery["Ranking of Bargain"]+grocery["Density of Grocery"]
df["Availability of Grocery|Bargain"]=ranking(df["Availability of Grocery|Bargain"])

pharmacy part

In [101]:
pha=pd.read_csv("../Daily Needs/pharmacy_zips.csv")
pha["Zip Code"]=pha["Zip Code"].astype(str)

In [102]:
dens=[]
for i in range(len(zip_code)):
    dens.append(pha[pha["Zip Code"]==zip_code[i]]["Zip Code"].count())

In [103]:
df["Availability of Pharmacy"]=dens/df["Population"]
df["Availability of Pharmacy"]=ranking(df["Availability of Pharmacy"])

the safety part

In [104]:
crim=pd.read_csv("../Safety/crime_dc.csv")
crim=crim.dropna(subset=["Zip Code"])
crim["Zip Code"]=crim["Zip Code"].astype(int)
crim["Zip Code"]=crim["Zip Code"].astype(str)

In [105]:
cri=pd.DataFrame(zip_code)
pro_t_o=[]
pro_b=[]
pro_t_f=[]
pro_m=[]
pro_a=[]
vio_h=[]
vio_a=[]
vio_r=[]
vio_s=[]
for i in range(len(zip_code)):
    pro_t_o.append((crim[(crim["Zip Code"]==zip_code[i]) & (crim["offensekey"]=="property|theft/other")])["offensekey"].count())
    pro_b.append((crim[(crim["Zip Code"]==zip_code[i]) & (crim["offensekey"]=="property|burglary")])["offensekey"].count())
    pro_t_f.append((crim[(crim["Zip Code"]==zip_code[i]) & (crim["offensekey"]=="property|theft f/auto")])["offensekey"].count())
    pro_m.append((crim[(crim["Zip Code"]==zip_code[i]) & (crim["offensekey"]=="property|motor vehicle theft")])["offensekey"].count())
    pro_a.append((crim[(crim["Zip Code"]==zip_code[i]) & (crim["offensekey"]=="property|arson")])["offensekey"].count())
    vio_h.append((crim[(crim["Zip Code"]==zip_code[i]) & (crim["offensekey"]=="violent|homicide")])["offensekey"].count())
    vio_a.append((crim[(crim["Zip Code"]==zip_code[i]) & (crim["offensekey"]=="violent|assault w/dangerous weapon")])["offensekey"].count())
    vio_r.append((crim[(crim["Zip Code"]==zip_code[i]) & (crim["offensekey"]=="violent|robbery")])["offensekey"].count())
    vio_s.append((crim[(crim["Zip Code"]==zip_code[i]) & (crim["offensekey"]=="violent|sex abuse")])["offensekey"].count())  

In [106]:
cri=cri.rename(columns={0:'Zip Code'})

In [107]:
cri["property|theft/other"]=pro_t_o
cri["property|theft/other"]=ranking(-cri["property|theft/other"])
cri["property|burglary"]=pro_b
cri["property|burglary"]=ranking(-cri["property|burglary"])
cri["property|theft f/auto"]=pro_t_f
cri["property|theft f/auto"]=ranking(-cri["property|theft f/auto"])
cri["property|motor vehicle theft"]=pro_m
cri["property|motor vehicle theft"]=ranking(-cri["property|motor vehicle theft"])
cri["property|arson"]=pro_a
cri["property|arson"]=ranking(-cri["property|arson"])
cri["violent|homicide"]=vio_h
cri["violent|homicide"]=ranking(-cri["violent|homicide"])
cri["violent|assault w/dangerous weapon"]=vio_a
cri["violent|assault w/dangerous weapon"]=ranking(-cri["violent|assault w/dangerous weapon"])
cri["violent|robbery"]=vio_r
cri["violent|robbery"]=ranking(-cri["violent|robbery"])
cri["violent|sex abuse"]=vio_s
cri["violent|sex abuse"]=ranking(-cri["violent|sex abuse"])

In [108]:
cri["Property"]=ranking(cri["property|theft/other"]+cri["property|burglary"]++cri["property|theft f/auto"]+
                       cri["property|motor vehicle theft"]+cri["property|arson"])

In [109]:
cri["Violent"]=ranking(cri["violent|homicide"]+cri["violent|assault w/dangerous weapon"]+cri["violent|robbery"]
                       +cri["violent|sex abuse"])

In [110]:
cri["Performance of Crime"]=ranking(cri["Property"]+cri["Violent"])

In [111]:
df=df.merge(cri,how="outer",on="Zip Code")

park part

In [112]:
park=pd.read_csv("../Leisure/Parks_zips.csv")

In [113]:
park=pd.DataFrame(park)
park["Zip Code"]=park["Zip Code"].astype(str)
dens=[]
for i in range(len(zip_code)):
    dens.append(park[park["Zip Code"]==zip_code[i]]["Zip Code"].count())

In [114]:
df["Availability of Parks"]=dens/df["Population"]
df["Availability of Parks"]=ranking(df["Availability of Parks"])

national park part

In [115]:
napa=pd.read_csv("../Leisure/National_Parks_zips.csv",dtype={"Zip Code":str})


In [116]:
mark=[]
for i in range(len(napa)-1):
    if (napa.iloc[i+1]["NAME"]==napa.iloc[i]["NAME"]):
        if(napa.iloc[i+1]["Zip Code"]==napa.iloc[i]["Zip Code"]):
            mark.append("0")
        else:
            mark.append(napa.iloc[i]["Zip Code"])
    else:
        mark.append(napa.iloc[i]["Zip Code"])

In [117]:
mark.append(napa.iloc[-1]["Zip Code"])

In [118]:
napa["MARK"]=mark

In [119]:
dens2=[]
for i in range(len(zip_code)):
    dens2.append(napa[napa["MARK"]==zip_code[i]]["MARK"].count())

In [120]:
df["Availability of National Parks"]=dens2/df["Population"]
df["Availability of National Parks"]=ranking(df["Availability of National Parks"])

sports part

In [121]:
spo=pd.read_csv("../Leisure/sports.csv",header=None)

In [122]:
spo_name=spo[::5]
spo_add=spo[3::5]

In [123]:
spo_zip=[]
for i in range(len(spo_add)):
    spo_zip.append(str.split(str.split(spo_add[0].iloc[i],sep=",")[-1].replace(" ",""),sep="-")[0])
    spo_zip

In [124]:
sport=pd.DataFrame(spo_name)
sport=sport.rename(columns={0:"Names"})
sport["Zip Code"]=spo_zip
sport["Zip Code"]=sport["Zip Code"].astype(str)
dens=[]
for i in range(len(zip_code)):
    dens.append(sport[sport["Zip Code"]==zip_code[i]]["Names"].count())

In [125]:
df["Availability of Sport Facility"]=dens/df["Population"]
df["Availability of Sport Facility"]=ranking(df["Availability of Sport Facility"])

restaurant part

In [126]:
cafe=pd.read_csv("../Daily Needs/cafe.csv")
bake=pd.read_csv("../Daily Needs/cafe.csv")
bar=pd.read_csv("../Daily Needs/bar.csv")
deli=pd.read_csv("../Daily Needs/delivery.csv")
other=pd.read_csv("../Daily Needs/other.csv")
cof=pd.read_csv("../Daily Needs/Coffee.csv")

In [127]:
cafe=pd.concat([cafe,cof])

C:\Users\19018\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [128]:
rest=pd.DataFrame(zip_code)
rest=rest.rename(columns={0:"Zip Code"})

In [129]:
dens=[]
ave=[]
price=[]
cafe["Zip Code"]=cafe["Zip Code"].astype(str)
for i in range(len(zip_code)):
    dens.append(cafe[cafe["Zip Code"]==zip_code[i]]["id"].count())
    ave.append(cafe[cafe["Zip Code"]==zip_code[i]]["rating"].mean())
    price.append(cafe[cafe["Zip Code"]==zip_code[i]]["price_level"].mean())
rest["Availability of Cafe"]=dens/df["Population"]
rest["Availability of Cafe"]=ranking(rest["Availability of Cafe"])
rest["Rating of Cafe"]=ave
rest["Rating of Cafe"]=ranking(rest["Rating of Cafe"])
rest["Price level of Cafe"]=price
rest["Price level of Cafe"]=ranking(-rest["Price level of Cafe"])
rest["Ranking of Cafe"]=ranking(rest["Price level of Cafe"]+rest["Rating of Cafe"]+rest["Availability of Cafe"])

In [130]:
dens=[]
ave=[]
price=[]
bar["Zip Code"]=bar["Zip Code"].astype(str)
for i in range(len(zip_code)):
    dens.append(bar[bar["Zip Code"]==zip_code[i]]["id"].count())
    ave.append(bar[bar["Zip Code"]==zip_code[i]]["rating"].mean())
    price.append(bar[bar["Zip Code"]==zip_code[i]]["price_level"].mean())
rest["Availability of Bar"]=dens/df["Population"]
rest["Availability of Bar"]=ranking(rest["Availability of Bar"])
rest["Rating of Bar"]=ave
rest["Rating of Bar"]=ranking(rest["Rating of Bar"])
rest["Price level of Bar"]=price
rest["Price level of Bar"]=ranking(-rest["Price level of Bar"])
rest["Ranking of Bar"]=ranking(rest["Price level of Bar"]+rest["Rating of Bar"]+rest["Availability of Bar"])

In [131]:
dens=[]
ave=[]
price=[]
bake["Zip Code"]=bake["Zip Code"].astype(str)
for i in range(len(zip_code)):
    dens.append(bake[bake["Zip Code"]==zip_code[i]]["id"].count())
    ave.append(bake[bake["Zip Code"]==zip_code[i]]["rating"].mean())
    price.append(bake[bake["Zip Code"]==zip_code[i]]["price_level"].mean())
rest["Availability of Bakery"]=dens/df["Population"]
rest["Availability of Bakery"]=ranking(rest["Availability of Bakery"])
rest["Rating of Bakery"]=ave
rest["Rating of Bakery"]=ranking(rest["Rating of Bakery"])
rest["Price level of Bakery"]=price
rest["Price level of Bakery"]=ranking(-rest["Price level of Bakery"])
rest["Ranking of Bakery"]=ranking(rest["Price level of Bakery"]+rest["Rating of Bakery"]+rest["Availability of Bakery"])

In [132]:
dens=[]
ave=[]
price=[]
deli["Zip Code"]=deli["Zip Code"].astype(str)
for i in range(len(zip_code)):
    dens.append(deli[deli["Zip Code"]==zip_code[i]]["id"].count())
    ave.append(deli[deli["Zip Code"]==zip_code[i]]["rating"].mean())
    price.append(deli[deli["Zip Code"]==zip_code[i]]["price_level"].mean())
rest["Availability of Delivery Restaurant"]=dens/df["Population"]
rest["Availability of Delivery Restaurant"]=ranking(rest["Availability of Delivery Restaurant"])
rest["Rating of Delivery Restaurant"]=ave
rest["Rating of Delivery Restaurant"]=ranking(rest["Rating of Delivery Restaurant"])
rest["Price level of Delivery Restaurant"]=price
rest["Price level of Delivery Restaurant"]=ranking(-rest["Price level of Delivery Restaurant"])
rest["Ranking of Delivery Restaurant"]=ranking(rest["Price level of Delivery Restaurant"]+
                                               rest["Rating of Delivery Restaurant"]+
                                               rest["Availability of Delivery Restaurant"])

In [133]:
dens=[]
ave=[]
price=[]
other["Zip Code"]=other["Zip Code"].astype(str)
for i in range(len(zip_code)):
    dens.append(other[other["Zip Code"]==zip_code[i]]["id"].count())
    ave.append(other[other["Zip Code"]==zip_code[i]]["rating"].mean())
    price.append(other[other["Zip Code"]==zip_code[i]]["price_level"].mean())
rest["Availability of Other Restaurant"]=dens/df["Population"]
rest["Availability of Other Restaurant"]=ranking(rest["Availability of Other Restaurant"])
rest["Rating of Other Restaurant"]=ave
rest["Rating of Other Restaurant"]=ranking(rest["Rating of Other Restaurant"])
rest["Price level of Other Restaurant"]=price
rest["Price level of Other Restaurant"]=ranking(-rest["Price level of Other Restaurant"])
rest["Ranking of Other Restaurant"]=ranking(rest["Price level of Other Restaurant"]+
                                            rest["Rating of Other Restaurant"]+
                                            rest["Availability of Other Restaurant"])

In [134]:
df=df.merge(rest,on="Zip Code",how="left")

In [135]:
dr=pd.read_csv("Driving Area 7 am.csv")
cy=pd.read_csv("Cycling Area 7 am.csv")

In [136]:
df["Driving at 7 am|10 mins"]=ranking(dr["Area of 10 mins"])
df["Driving at 7 am|30 mins"]=ranking(dr["Area of 30 mins"])
df["Driving at 7 am|60 mins"]=ranking(dr["Area of 60 mins"])

In [137]:
df["Cycling at 7 am|10 mins"]=ranking(cy["Area of 10 mins"])
df["Cycling at 7 am|30 mins"]=ranking(cy["Area of 30 mins"])
df["Cycling at 7 am|60 mins"]=ranking(cy["Area of 60 mins"])

In [138]:
df["Driving at 7 am"]=ranking(df["Driving at 7 am|10 mins"]+df["Driving at 7 am|30 mins"]+df["Driving at 7 am|60 mins"])
df["Cycling at 7 am"]=ranking(df["Cycling at 7 am|10 mins"]+df["Cycling at 7 am|30 mins"]+df["Cycling at 7 am|60 mins"])

In [139]:
dr2=pd.read_csv("Driving Area 5 pm.csv")
cy2=pd.read_csv("Cycling Area 5 pm.csv")

In [140]:
df["Driving at 5 pm|10 mins"]=ranking(dr2["Area of 10 mins"])
df["Driving at 5 pm|30 mins"]=ranking(dr2["Area of 30 mins"])
df["Driving at 5 pm|60 mins"]=ranking(dr2["Area of 60 mins"])

In [141]:
df["Cycling at 5 pm|10 mins"]=ranking(cy2["Area of 10 mins"])
df["Cycling at 5 pm|30 mins"]=ranking(cy2["Area of 30 mins"])
df["Cycling at 5 pm|60 mins"]=ranking(cy2["Area of 60 mins"])

In [142]:
df["Driving at 5 pm"]=ranking(df["Driving at 5 pm|10 mins"]+df["Driving at 5 pm|30 mins"]+df["Driving at 5 pm|60 mins"])
df["Cycling at 5 pm"]=ranking(df["Cycling at 5 pm|10 mins"]+df["Cycling at 5 pm|30 mins"]+df["Cycling at 5 pm|60 mins"])

In [143]:
data=df.to_json()

In [144]:
file = open('data.json','w',encoding='utf-8')
json.dump(data,file,ensure_ascii=False)

In [145]:
df.to_csv("data.csv")

In [146]:
df

,Zip Code,Population,Price for Sale|Studio,Price for Sale|1 Bed,Price for Sale|2 Beds,Price for Sale|3 Beds,Price for Sale|4 Beds,Price for Sale|5 Beds,Price for Sale|6 Beds,Price for Sale|7 Beds,...,Driving at 7 am,Cycling at 7 am,Driving at 5 pm|10 mins,Driving at 5 pm|30 mins,Driving at 5 pm|60 mins,Cycling at 5 pm|10 mins,Cycling at 5 pm|30 mins,Cycling at 5 pm|60 mins,Driving at 5 pm,Cycling at 5 pm
0,20001,38551.0,335000.000000,469753.959184,734827.966102,8.822890e+05,1.155034e+06,1.159850e+06,1.699500e+06,NaN,...,2,3,2,3,3,2,3,3,2,3
1,20002,52370.0,NaN,391722.535714,539249.919540,7.913266e+05,9.924978e+05,1.220625e+06,1.435000e+06,NaN,...,2,4,3,4,1,3,3,4,2,4
2,20003,26454.0,368283.333333,432204.068966,747956.897436,8.834842e+05,1.238833e+06,1.916000e+06,NaN,NaN,...,5,2,4,5,5,3,1,2,5,2
3,20004,12775.0,NaN,463845.454545,663000.000000,8.326667e+05,NaN,NaN,NaN,NaN,...,5,3,3,5,5,2,2,4,5,3
4,20005,1622.0,257200.000000,484829.230769,710192.592593,1.137500e+06,NaN,2.970000e+06,NaN,NaN,...,3,5,2,3,4,5,4,5,3,5
5,20006,3227.0,258333.333333,380000.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,4,3,2,4,5,2,3,3,4,3
6,20007,26866.0,200112.500000,360688.080000,943500.000000,1.229389e+06,1.895678e+06,2.798633e+06,4.372859e+06,2470000.0,...,1,1,1,1,3,1,1,2,1,1
7,20008,27525.0,218883.333333,372102.302326,665056.333333,1.406083e+06,1.502312e+06,2.125363e+06,1.550000e+06,NaN,...,3,5,4,2,4,4,5,5,3,5
8,20009,47992.0,272402.055556,430932.090909,698360.353448,1.147511e+06,1.508111e+06,1.300300e+06,2.537500e+06,1699999.0,...,1,5,1,1,1,5,4,5,1,5
9,20010,30138.0,236300.000000,353014.285714,590880.000000,7.920343e+05,8.639167e+05,1.385000e+06,1.132250e+06,1510000.0,...,1,5,1,1,1,4,5,4,1,5
